In [15]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.optimize import curve_fit

from sklearn.linear_model import LinearRegression
from data_utils import load_data, mission_day_to_year
from modeling import compute_exposure

In [11]:
data_dir = "./data"
virgo_file = "VIRGO_Level1.txt"
results_dir = os.path.join(data_dir, "analysis")

data = load_data(os.path.join(data_dir, virgo_file))

t = data["timestamp"]  # Time in mission days
pmo_a = data["pmo6v_a"]
pmo_b = data["pmo6v_b"]
temp = data["temperature"]

In [13]:
mean_b = pmo_b.mean()
# data["exposure_a"] = compute_exposure(t, pmo_a)
# data["exposure_b"] = compute_exposure(t, pmo_b)
data["exposure_a"] = compute_exposure(t, pmo_a, "exposure_sum", mean_b)
data["exposure_b"] = compute_exposure(t, pmo_b, "exposure_sum", mean_b)

data_nn = data[["timestamp", "pmo6v_a", "pmo6v_b", "exposure_a", "exposure_b"]].dropna()
t_nn = data_nn["timestamp"].values
e_a_nn = data_nn["exposure_a"].values
e_b_nn = data_nn["exposure_b"].values
x_a_nn = data_nn["pmo6v_a"].values
x_b_nn = data_nn["pmo6v_b"].values

ratio_a_b = x_a_nn/x_b_nn

plt.figure(1, figsize=(16, 8))
plt.plot(t_nn, ratio_a_b)
plt.savefig(os.path.join(results_dir, "ratio_a_b_raw.pdf"), bbox_inches="tight", quality=100, dpi=200)
plt.show()

print("min ratio_a_b", ratio_a_b.min())

# data.sample(15, random_state=2)

TypeError: compute_exposure() takes from 1 to 3 positional arguments but 4 were given

In [ ]:
epsilon = 1e-5
gamma = ratio_a_b.min()

Y = np.log(ratio_a_b - gamma + epsilon)
X = e_a_nn.reshape(-1, 1)

regressor = LinearRegression(fit_intercept=True)
regressor.fit(X, Y)

a = regressor.coef_[0]
b = regressor.intercept_

lambda_ = -a
t_0 = b / lambda_

ratio_a_b_pred = gamma + np.exp(-lambda_ * (e_a_nn - t_0))


def exponential_linear_model(x, gamma, lambda_, t_0, linear):
    return np.exp(-lambda_ * (x - t_0)) + gamma + linear * x

def exponential_model(x, gamma, lambda_, t_0):
    return np.exp(-lambda_ * (x - t_0)) + gamma

popt, pcov = curve_fit(exponential_model,  e_a_nn,  ratio_a_b, p0=(gamma, lambda_, t_0))
popt_lin, pcov = curve_fit(exponential_linear_model,  e_a_nn,  ratio_a_b, p0=(gamma, lambda_, t_0, 0))


plt.figure(2, figsize=(16, 8))
# plt.plot(t_nn, ratio_a_b, t_nn, ratio_a_b_pred, t_nn, exponential_model(e_a_nn, *popt))
plt.plot(t_nn, ratio_a_b_pred, t_nn, exponential_model(e_a_nn, *popt), 
         t_nn, exponential_linear_model(e_a_nn, *popt_lin))
plt.show()

In [ ]:
x_a_nn_c = x_a_nn / exponential_model(e_a_nn, *popt)
x_a_nn_c_lin = x_a_nn / exponential_linear_model(e_a_nn, *popt_lin)

x_b_nn_c = x_b_nn / exponential_model(e_b_nn, *popt)
x_b_nn_c_lin = x_b_nn / exponential_linear_model(e_b_nn, *popt_lin)


plt.figure(3, figsize=(16, 24))
plt.subplot(4, 1, 1)
plt.plot(t_nn, x_a_nn, t_nn, x_b_nn, t_nn, x_a_nn_c, t_nn, x_a_nn_c_lin)
plt.subplot(4, 1, 2)
plt.plot(t_nn, x_b_nn, t_nn, x_a_nn_c)
plt.subplot(4, 1, 3)
plt.plot(t_nn, x_b_nn, t_nn, x_a_nn_c_lin)
plt.subplot(4, 1, 4)
plt.plot(t_nn, x_b_nn, t_nn, x_a_nn_c_lin, t_nn, x_b_nn_c_lin)
plt.legend(["pmo_b", "pmo_a_c_lin", "pmo_b_c_lin"])
plt.show()

In [9]:

N = 15
ratio = ratio_a_b
x_b_nn_c_lin = x_b_nn

popt = np.zeros(shape=(3,))
for i in range(1, N+1):
    ratio = x_a_nn / x_b_nn_c_lin
    
    popt_lin, _ = curve_fit(exponential_linear_model,  e_a_nn,  ratio, p0=(lambda_, t_0, 0))
    x_a_nn_c_lin = x_a_nn / exponential_linear_model(e_a_nn, *popt_lin)
    x_b_nn_c_lin = x_b_nn / exponential_linear_model(e_b_nn, *popt_lin)


#     popt_lin, _ = curve_fit(exponential_model,  e_a_nn,  ratio, p0=(lambda_, t_0))
#     x_a_nn_c_lin = x_a_nn / exponential_model(e_a_nn, *popt_lin)
#     x_b_nn_c_lin = x_b_nn / exponential_model(e_b_nn, *popt_lin)

    print("norm change", np.linalg.norm(popt - popt_lin))
    popt = popt_lin
    
#     plt.figure(4, figsize=(16, N * 4))
#     plt.subplot(N, 2, 2*(i-1)+1)
#     plt.plot(t_nn, x_b_nn, t_nn, x_a_nn_c_lin, t_nn, x_b_nn_c_lin)
#     plt.legend(["pmo_b", "pmo_a_c_lin", "pmo_b_c_lin"])
#     plt.subplot(N, 2, 2*(i-1)+2)
#     plt.plot(t_nn, ratio_a_b, t_nn, ratio)
#     plt.legend(["ratio_a_b", "ratio"])
    
#     plt.figure(4, figsize=(16, N * 8))
#     plt.subplot(2*N, 1, 2*(i-1)+1)
#     plt.plot(t_nn, x_b_nn, t_nn, x_a_nn_c_lin, t_nn, x_b_nn_c_lin)
#     plt.legend(["pmo_b", "pmo_a_c_lin", "pmo_b_c_lin"])
#     plt.subplot(2*N, 1, 2*(i-1)+2)
#     plt.plot(t_nn, ratio_a_b, t_nn, ratio, t_nn, exponential_linear_model(e_a_nn, *popt))
#     plt.legend(["ratio_a_b", "ratio"])

plt.show()


NameError: name 'ratio_a_b' is not defined

In [20]:
np.array(list(map(mission_day_to_year, t[:100])))

array([1996.13148802, 1996.13149185, 1996.13149377, 1996.13149952,
       1996.13150308, 1996.131505  , 1996.13151075, 1996.13151458,
       1996.1315165 , 1996.13152225, 1996.13152608, 1996.13152799,
       1996.13153374, 1996.1315373 , 1996.13153922, 1996.13154497,
       1996.1315488 , 1996.13155072, 1996.13155647, 1996.1315603 ,
       1996.13156222, 1996.13156797, 1996.13157153, 1996.13157344,
       1996.13157919, 1996.13158303, 1996.13158494, 1996.13159069,
       1996.13159452, 1996.13159644, 1996.13160219, 1996.13160575,
       1996.13160767, 1996.13161342, 1996.13161725, 1996.13161916,
       1996.13162491, 1996.13162875, 1996.13163066, 1996.13163641,
       1996.13163997, 1996.13164189, 1996.13164764, 1996.13165147,
       1996.13165339, 1996.13165914, 1996.13166297, 1996.13166489,
       1996.13167064, 1996.1316742 , 1996.13167611, 1996.13168186,
       1996.13168569, 1996.13168761, 1996.13169336, 1996.13169719,
       1996.13169911, 1996.13170486, 1996.13170842, 1996.13171